In [6]:
#Importando pacotes utilizados
import pyautogui
import time
import pyperclip
import pandas as pd

In [7]:
#Leitura dos CEPs

#Esses são os ceps de todos os estabelecimentos
ceps = pd.read_csv("ceps_estab.csv")

In [8]:
#Dps de eliminar os duplicados cai de 252 mil para 6870
ceps.shape

(6870, 2)

In [9]:
ceps = ceps[["CEP Estab"]]
ceps

,CEP Estab
0,55780000
1,50010240
2,55016080
3,55002200
4,53130410
...,...
6865,53110170
6866,54350110
6867,53220120
6868,55195644


### Buscando coordenadas

In [10]:
pyautogui.PAUSE = 0.2
pyautogui.alert("Vai começar, aperte OK e não mexa em nada")
lat_long = []
start= 0  #defina o CEP de início
stop= 10 #defina o CEP final
bloco = 10 #numero de abas abertas por vez

#Abre uma nova janela
pyautogui.hotkey('ctrl', 'n')

for j in range(start,stop,bloco):
    
    #loop para abrir as abas e pesquisar urls
    for i in range(j,j+bloco):
        cep = ceps["CEP Estab"].loc[i] #selecionando coluna que possui o CEP
        uri = 'https://www.google.com/maps/search/' #adicionei search para modificar
        url = uri+str(cep)
        pyautogui.hotkey('ctrl', 't')
        pyperclip.copy(url)
        pyautogui.hotkey("ctrl", "v")
        
        pyautogui.press("enter")

    pyautogui.hotkey('ctrl', '2')
    time.sleep(15)
    

    #Loop para fechar abas e coletar as urls
    for i in range(j,j+bloco):
        time.sleep(2)
        pyautogui.click(500, 50) #ajustando a posição do mouse para copiar urls
        pyautogui.hotkey("ctrl", "c")
        lat_long.append(pyperclip.paste())
        pyautogui.hotkey("ctrl", "w")   

#Organizando ceps coletados em um data frame        
df_lat_long = pd.DataFrame.from_dict(lat_long)
ceps = ceps[start:stop].reset_index()
del ceps["index"]
df_lat_long.insert(0,"cep", ceps, True)
df_lat_long

#Salvando coordenadas em um arquivo CSV
#df_lat_long.to_csv(f"cep_url_{start}_{stop}.csv")

,cep,0
0,55780000,https://www.google.com/maps/place/Frei+Migueli...
1,50010240,https://www.google.com/maps/place/R.+do+Impera...
2,55016080,https://www.google.com/maps/place/R.+Tupy+-+Sa...
3,55002200,https://www.google.com/maps/place/R.+Vig.+Frei...
4,53130410,https://www.google.com/maps/place/Av.+Dr.+Jos%...
5,50030000,https://www.google.com/maps/place/Av.+Marqu%C3...
6,54310310,https://www.google.com/maps/place/Av.+Barreto+...
7,53010230,https://www.google.com/maps/place/Av.+Santos+D...
8,55125000,https://www.google.com/maps/place/Toritama+-+P...
9,55590000,https://www.google.com/maps/place/Ipojuca+-+PE...


In [11]:
base_ceps = df_lat_long

### Limpando Lat e Long

In [12]:
start= 0  #defina o CEP de início
stop= 10 #defina o CEP final

lat_long = []
for i in range(start,stop):
    texto = base_ceps.iloc[i,1]
    indice_final = texto.find("/data=")
    if indice_final == -1:
        lat_long_i = {"lat":("-"), "long":"-"}
        lat_long.append(lat_long_i)
    else:    
        indice_inicial = texto.find("/@")
        texto = texto[indice_inicial+2:indice_final-4]
        texto = texto.split(",")
        lat = texto[0]
        long = texto[1]
        lat_long_i = {"lat":lat, "long":long}
        lat_long.append(lat_long_i)
        
df_lat_long = pd.DataFrame.from_dict(lat_long)    

In [13]:
df_lat_long.insert(0,"cep", base_ceps[["cep"]],True)
df_lat_long 

,cep,long,lat
0,55780000,-35.886879,-7.9311086
1,50010240,-34.8769008,-8.0630207
2,55016080,-35.9655839,-8.2772306
3,55002200,-35.9682904,-8.2854148
4,53130410,-34.8371604,-7.9853241
5,50030000,-34.8729745,-8.0633339
6,54310310,-34.926085,-8.1630845
7,53010230,-34.8536215,-8.02133
8,55125000,-36.0595888,-7.9945535
9,55590000,-35.094164,-8.4578219
